In [ ]:
import cv2 as cv
import numpy as np
import pandas as pd
from glob import glob
from sklearn.utils import shuffle

from config import efficientNet_config
from models import efficientNetV2B0_model, efficientNetV2B3_model, vgg16_model, resnet50_model, mobilenetv2_model,frank_model
from utils import plt_heatmap, plt_roccurve

from sklearn.metrics import confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import roc_curve, auc, accuracy_score, f1_score
from sklearn.preprocessing import OneHotEncoder

import os
# 使用第一張 GPU 卡
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
tes_df = pd.read_csv("../dataset/test_dataset/direc/new_class.csv", index_col=False)
cls    = len(set(tes_df['new_class'].values))

# 載入模型

In [ ]:
model = efficientNetV2B0_model(cls)  # 類別
model.load_weights('../model/202211/20221130.weights')

In [ ]:
height  = efficientNet_config['height_B0']
width   = efficientNet_config['width_B0']

In [ ]:
# label
# tes_df = pd.read_csv("../dataset/test_dataset/direc/test_new_class_20221129.csv", index_col=False)

tes_img_labels = tes_df['new_class'].values
print("真值")
print(tes_img_labels)


# label one hot
# tes_img_labels_onehot  = OneHotEncoder().fit_transform(tes_img_labels).toarray()

In [ ]:
tes_img_name_list = tes_df['img_name'].values.tolist()
print('test number = '  , len(tes_img_name_list))

In [ ]:
# image array
tes_dir = '../dataset/test_dataset/img/CropSmooth/'
# npy
# tes_dir = '../dataset/test_dataset/img/Crop_Edge/'

tes_img_arrays = []
## test image array
for img_name in tes_img_name_list:
    # image
    img_array = cv.imread(tes_dir+img_name)
    img_array = cv.resize(img_array,(height,width))     # resize to (224,224)
    tes_img_arrays.append(img_array)

    # npy
    # img_array = np.load(tes_dir + img_name.split(".")[0] + ".npy")
    # img_array = cv.resize(img_array,(height,width))
    # tes_img_arrays.append(img_array)


In [ ]:
tes_img_arrays = np.array(tes_img_arrays)
# tes_img_arrays = np.repeat(tes_img_arrays[..., np.newaxis], 3, -1)   # 灰階轉RGB

In [ ]:
print('測試集維度= ',tes_img_arrays.shape)
print('測試集label維度= ',tes_img_labels.shape)

# 打亂

In [ ]:
# tes_img_arrays, tes_img_labels = shuffle(tes_img_arrays, tes_img_labels)

In [ ]:
# tes_img_arrays.shape

# 預測

In [ ]:
pred_result = model.predict(tes_img_arrays)

In [ ]:
pred_result_1d = []
for p in pred_result:
    p_cls = np.argmax(p)
    # print(p_cls, p)
    pred_result_1d.append(p_cls)

for i in range(len(tes_img_labels)):
    print('t=', tes_img_labels[i], 'p=',pred_result_1d[i])

# 結果

In [ ]:
# fpr, tpr, thresholds = roc_curve(tes_img_labels, pred_result, pos_label=1) 
# roc_auc = auc(fpr, tpr)
# plt_roccurve(fpr,tpr,roc_auc)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
def plt_heatmap_3c(con_ma):
    '''
    :input: con_ma:3*3
    '''
    sns.set(font_scale=2)
    f,ax=plt.subplots()
    sns.heatmap(con_ma, annot=True, ax=ax, fmt='d', cmap='YlGnBu', vmax=25, vmin=0) # 畫熱力圖
    ax.set_title('confusion matrix') # 標題
    ax.set_xlabel('predict')         # x軸
    ax.set_ylabel('true')            # y軸
    # ax.xaxis.set_ticklabels(['0', '1', '2'])
    # ax.yaxis.set_ticklabels(['0', '1', '2'])
    plt.show()

In [ ]:
cm = confusion_matrix(tes_img_labels, pred_result_1d)#.ravel()


cm_df = pd.DataFrame(cm,)
cm_df
plt_heatmap_3c(cm_df)

# sensitivity = round(tp / (tp+fn), 4)
# specificity = round(tn / (tn+fp), 4)
# balanced_acc= round((sensitivity+specificity)/2, 4)
# precision   = round(tp / (tp+fp), 4)
# f1score     = round(2/((1/precision)+(1/sensitivity)), 4)
# accuracy    = round((tp+tn)/(tn+fp+fn+tp), 4)

# print('Sensitivity= ',sensitivity)
# print('Specificity= ',specificity)
# print('Balanced_acc= ',balanced_acc)
# print('Precision= ', precision)
# print('f1score= ', f1score)
# print('Accuracy= ',accuracy)

In [ ]:
accuracy_score(tes_img_labels, pred_result_1d)

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import roc_curve, auc, accuracy_score, f1_score

In [ ]:
CM3 = multilabel_confusion_matrix(tes_img_labels, pred_result_1d,labels=[0, 1, 2])

In [ ]:
for CM in CM3:
    print(CM)

In [ ]:
print('MACRO F1 SCORE =', f1_score(tes_img_labels, pred_result_1d, average='macro'))
print('MICRO F1 SCORE =', f1_score(tes_img_labels, pred_result_1d, average='micro'))

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
res = []
for l in [0,1,2]:
     prec,recall,_,_ = precision_recall_fscore_support(np.array(tes_img_labels)==l,
                                                       np.array(pred_result_1d)==l,
                                                       pos_label=True,average=None)
     res.append([l, recall[0], recall[1]])

pd.DataFrame(res,columns = ['class','sensitivity','specificity'])

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(tes_img_labels, pred_result_1d, labels=[0, 1, 2]))